# Analyzing a player's shot selection after going on a shooting streak.

In [26]:
# import packages
from nba_api.stats.endpoints import shotchartdetail
import json
import requests
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# Grab player and team ids
# Load teams file
teams = json.loads(requests.get('https://raw.githubusercontent.com/bttmly/nba/master/data/teams.json').text)
# Load players file
players = json.loads(requests.get('https://lifeislinear.davidson.edu/sports/player_dicts.json').text)

### ID helper functions

In [24]:
# Get team ID based on team name
def get_team_id(teamInput):
    for team in teams:
        if team['teamName'] == teamInput:
            return team['teamId']
    return -1
# Get player ID based on player name
def get_player_id(first, last):
    for player in players:
        if player['firstName'] == first and player['lastName'] == last:
            return player['playerId']
    return -1
# Get Name based on player id
def get_player_name(playerId):
    for player in players:
        if player['playerId']==playerId:
            return player['firstName'], player['lastName']
    return -1

### Shooting percentage helper functions.

In [3]:
# Shooting percentage in each zone
def shooting_perc(group):
    made = group['SHOT_MADE_FLAG'].sum()
    attempted = group['SHOT_ATTEMPTED_FLAG'].sum()
    return round((made / attempted) * 100, 2)

# The percentage of total shots from each zone
def shot_perc(group, total):
    attempted = group['SHOT_ATTEMPTED_FLAG'].sum() # shots attempted in the zone
    return round((attempted / total) * 100, 2)

### Shot streak helper functions.

In [4]:
def find_hh(data, hh_thresh, shot_type=-1):
    # Data - the data frame to filter
    # player_id - the player we are investigating
    # hh_thresh - how many mae shots in a row to qualify for a "hot hand"

    if shot_type == 2:
        data = data[data["SHOT_TYPE"] == "2PT Field Goal"]
    elif shot_type == 3:
        data = data[data["SHOT_TYPE"] == "3PT Field Goal"]
    
    made_count = 0
    for index, row in data.iterrows():
        if row["SHOT_MADE_FLAG"] == 1:
            made_count += 1
        else:
            made_count = 0

        if made_count < hh_thresh:
            data.at[index, 'HOT_HAND'] = 0
        else:
            data.at[index, 'HOT_HAND'] = 1

    return data

def heat_check(data):
    # A function to find the misses following a hot hand
    
    # index - 1 was throwing errors, so to get the previous hot hand shots we will use shift
    temp = data
    temp['p_hh'] = data["HOT_HAND"].shift(1).fillna(0)
    data['HEAT_CHECK'] = ((data['SHOT_MADE_FLAG'] == 0) & (temp['p_hh'] == 1)).astype(int)
    
    return data

### Load the data

In [27]:
# Create JSON request
ahead_json = shotchartdetail.ShotChartDetail(
            # team_id = get_team_id('Los Angeles Lakers'),
            team_id = 0,
            player_id = get_player_id('LeBron', 'James'),
            context_measure_simple = 'FGA',
            season_nullable = '2022-23',
            season_type_all_star = ['Regular Season', 'Playoffs'],
            ahead_behind_nullable = 'Ahead or Tied')

# Load JSON data into dictionary
ahead_data = json.loads(ahead_json.get_json())

behind_json = shotchartdetail.ShotChartDetail(
            # team_id = get_team_id('Los Angeles Lakers'),
            team_id = 0,
            player_id = get_player_id('LeBron', 'James'),
            context_measure_simple = 'FGA',
            season_nullable = '2022-23',
            season_type_all_star = ['Regular Season', 'Playoffs'],
            ahead_behind_nullable = 'Behind or Tied')

behind_data = json.loads(behind_json.get_json())

# Convert the data into a data frame.
ahead_df = pd.concat(
    pd.DataFrame(shot, columns=ahead_data['resultSets'][0]['headers'])
    for shot in [ahead_data['resultSets'][0]['rowSet']]
)
behind_df = pd.concat(
    pd.DataFrame(shot, columns=behind_data['resultSets'][0]['headers'])
    for shot in [behind_data['resultSets'][0]['rowSet']]
)

### Preparing the data helper function.

In [40]:
STREAK_THRESH = 2

def prepare_data(df):
    # Find the player's shooting percentage per zone.
    df['ZONE_SPECIFIC'] = df['SHOT_ZONE_AREA'] + ' ' + df['SHOT_ZONE_RANGE']
    shooting_zones = df.groupby('ZONE_SPECIFIC')
    shooting_zones = shooting_zones.apply(shooting_perc)

    # Find the shot attempts when a player is on a streak.
    df = find_hh(df, STREAK_THRESH)
    
    # Find the player's misses after a streak. Then find the misses per zone.
    df = heat_check(df)

    # Keep only 3 point shots
    df = df[df['SHOT_TYPE'] == '3PT Field Goal']
    
    heat_check_zones = df.groupby('ZONE_SPECIFIC').sum()['HEAT_CHECK']

    # Convert the Series into data frames, merge, and then return the resulting analyis.
    shooting_zones = pd.DataFrame(shooting_zones, columns=['SHOOTING_PERCENTAGE'])
    heat_check_zones = pd.DataFrame(heat_check_zones)

    # Do we need to find the percentage of total shots taken in each zone?

    return pd.merge(shooting_zones, heat_check_zones, on='ZONE_SPECIFIC')



### Gather the appropriate data.

In [50]:
ahead_analysis = prepare_data(ahead_data)
behind_analysis = prepare_data(behind_data)

player_analysis = pd.merge(ahead_analysis, behind_analysis, on='ZONE_SPECIFIC', suffixes=('_AHEAD', '_BEHIND'))

In [51]:
player_analysis.head(14)

,SHOOTING_PERCENTAGE_AHEAD,HEAT_CHECK_AHEAD,SHOOTING_PERCENTAGE_BEHIND,HEAT_CHECK_BEHIND
ZONE_SPECIFIC,,,,
Center(C) 16-24 ft.,34.78,6,46.15,3
Center(C) 24+ ft.,32.43,19,32.31,12
Center(C) 8-16 ft.,50.00,4,52.17,5
Center(C) Less Than 8 ft.,67.12,16,65.45,26
Left Side Center(LC) 16-24 ft.,52.17,3,58.33,2
Left Side Center(LC) 24+ ft.,25.35,15,31.82,6
Left Side(L) 16-24 ft.,41.67,0,22.22,2
Left Side(L) 24+ ft.,33.33,0,12.50,2
Left Side(L) 8-16 ft.,34.38,4,18.52,3


### Iterate over the top 3 point shooters over the past years.

In [41]:
seasons = ['2023-24', '2022-23', '2021-22', '2020-21', '2019-20']
poi = ['LeBron James']

for p in poi:
    player_ahead = pd.DataFrame()
    player_behind = pd.DataFrame()
    p = p.split()
    
    for season in seasons:
        ahead_json = shotchartdetail.ShotChartDetail(
            # team_id = get_team_id('Los Angeles Lakers'),
            team_id = 0,
            player_id = get_player_id(p[0], p[1]),
            context_measure_simple = 'FGA',
            season_nullable = season,
            season_type_all_star = ['Regular Season', 'Playoffs'],
            ahead_behind_nullable = 'Ahead or Tied')

        # Load JSON data into dictionary
        ahead_data = json.loads(ahead_json.get_json())

        behind_json = shotchartdetail.ShotChartDetail(
                    # team_id = get_team_id('Los Angeles Lakers'),
                    team_id = 0,
                    player_id = get_player_id(p[0], p[1]),
                    context_measure_simple = 'FGA',
                    season_nullable = season,
                    season_type_all_star = ['Regular Season', 'Playoffs'],
                    ahead_behind_nullable = 'Behind or Tied')

        behind_data = json.loads(behind_json.get_json())

        # Convert the data into a data frame.
        ahead_df = pd.concat(
            pd.DataFrame(shot, columns=ahead_data['resultSets'][0]['headers'])
            for shot in [ahead_data['resultSets'][0]['rowSet']]
        )
        behind_df = pd.concat(
            pd.DataFrame(shot, columns=behind_data['resultSets'][0]['headers'])
            for shot in [behind_data['resultSets'][0]['rowSet']]
        )
        player_ahead = pd.concat([player_ahead, ahead_df], ignore_index=True)
        player_behind = pd.concat([player_behind, behind_df], ignore_index=True)

    ahead_analysis = prepare_data(player_ahead)
    behind_analysis = prepare_data(player_behind)

    player_analysis = pd.merge(ahead_analysis, behind_analysis, on='ZONE_SPECIFIC', suffixes=('_AHEAD', '_BEHIND'))

        

In [42]:
player_analysis.head(14)

,SHOOTING_PERCENTAGE_AHEAD,HEAT_CHECK_AHEAD,SHOOTING_PERCENTAGE_BEHIND,HEAT_CHECK_BEHIND
ZONE_SPECIFIC,,,,
Center(C) 24+ ft.,31.13,65,39.03,51
Left Side Center(LC) 24+ ft.,35.46,74,35.99,73
Left Side(L) 16-24 ft.,23.73,1,35.00,0
Left Side(L) 24+ ft.,48.57,3,28.57,12
Right Side Center(RC) 24+ ft.,39.38,47,34.27,50
Right Side(R) 24+ ft.,38.89,9,40.91,4
